### Problems: 
#### Longitudinal data of young children, we need to check if we take one entry each ? then we are only left with a few... 
#### subject ID as actually the match group (.xlsx)

In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy.io import mmread


In [3]:
#NOTE: There is a xlsx file in the folder that contains some info i try to integrate into the df´s
### NOTE: The first merged dataset here is not the one refered in the paper. the GSE2397987 one is found as the second one merged here... 
# Its a bit weird as the PBMC sample in GEO got only 86 samples and this one 119 (123) 


In [4]:
# what is disStage
### disease stage: paper specific
# what is Diff.From.Infant.Pos
### paper specific
# why is counts 119 and metadata 123 ?  
### See .xlsx file: 3 are excluded  

root_folder = '/var/datasets/single_papers/2023_Wimmers_CoVid_multi-omics/wimmers2023'

metadata = pd.read_csv(f"{root_folder}/bab_bulkRNA_metaData_220730.csv",sep=",")
print(metadata.shape)
metadata.head()


(123, 6)


,Subject.ID,PBMC.Collection.Date,Age at PBMC Collection (Months),Sex,Diff.From.Infant.Pos,disStage
0,CC0014,10/15/20,10.7,Male,-82.0,pre
1,CC0014,3/16/21,15.6,Male,70.0,conv
2,CC0016,12/15/20,12.8,Female,-58.0,pre
3,CC0016,7/1/21,19.3,Female,140.0,conv
4,CC0022,6/9/20,7.0,Female,-153.0,pre


In [5]:
rawdata = pd.read_csv(f"{root_folder}/raw_count_GRCh38_p22061_Florian_Analysis.txt",sep="\t", header=None).T
rawdata.columns = rawdata.iloc[0]
rawdata = rawdata.iloc[1:]
print(rawdata.shape)
rawdata.head()
# ID vol sample: 'p22061-s001_barcode269931-subjectidCC0069-iridCR141-date20200605' 


(119, 60696)


/tmp/ipykernel_126382/1491771001.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119) have mixed types. Specify dtype option on import or set low_memory=False.
  rawdata = pd.read_csv(f"{root_folder}/raw_count_GRCh38_p22061_Florian_Analysis.txt",sep="\t", header=None).T


,gene_id,ENSG00000223972,ENSG00000227232,ENSG00000278267,ENSG00000243485,ENSG00000284332,ENSG00000237613,ENSG00000268020,ENSG00000240361,ENSG00000186092,...,GU280_gp02_S,GU280_gp03_ORF3a,GU280_gp04_E,GU280_gp05_M,GU280_gp06_ORF6,GU280_gp07_ORF7a,GU280_gp08_ORF7b,GU280_gp09_ORF8,GU280_gp10_N,GU280_gp11_ORF10
1,p22061-s001_barcode269931-subjectidCC0069-irid...,0,28,30,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,p22061-s002_barcode270483-subjectidCC0022-irid...,0,4,21,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,p22061-s003_barcode288558-subjectidCC0138-irid...,1,6,14,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,p22061-s004_barcode322576-subjectidCC0256-irid...,3,11,15,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,p22061-s005_barcode327849-subjectidCC0259-irid...,3,37,12,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# Here i want to make it better readable what single entries mean (i split up that identifier string. .. )
columns_order = ['sample_id', 'barcode', 'subjectid', 'irid', 'date'] + list(rawdata.columns)
rawdata[['sample_id', 'barcode', 'subjectid', 'irid', 'date']] = rawdata['gene_id'].str.split('-', expand=True)
## add the first 4 chars of the barcode columns to the first one, as it is visible in the .xlsx file that it is supposed to be part of the sample_id
prefixes = rawdata['barcode'].str[:4]
rawdata['sample_id'] = rawdata['sample_id'] + '_' + prefixes
rawdata = rawdata[columns_order]
rawdata.head()

,sample_id,barcode,subjectid,irid,date,gene_id,ENSG00000223972,ENSG00000227232,ENSG00000278267,ENSG00000243485,...,GU280_gp02_S,GU280_gp03_ORF3a,GU280_gp04_E,GU280_gp05_M,GU280_gp06_ORF6,GU280_gp07_ORF7a,GU280_gp08_ORF7b,GU280_gp09_ORF8,GU280_gp10_N,GU280_gp11_ORF10
1,p22061_s001,s001_barcode269931,subjectidCC0069,iridCR141,date20200605,p22061-s001_barcode269931-subjectidCC0069-irid...,0,28,30,0,...,0,0,0,0,0,0,0,0,0,0
2,p22061_s002,s002_barcode270483,subjectidCC0022,iridCR074,date20200609,p22061-s002_barcode270483-subjectidCC0022-irid...,0,4,21,0,...,0,0,0,0,0,0,0,0,0,0
3,p22061_s003,s003_barcode288558,subjectidCC0138,iridCR069,date20200715,p22061-s003_barcode288558-subjectidCC0138-irid...,1,6,14,0,...,0,0,0,0,0,0,0,0,0,0
4,p22061_s004,s004_barcode322576,subjectidCC0256,iridCR169,date20200923,p22061-s004_barcode322576-subjectidCC0256-irid...,3,11,15,0,...,0,0,0,0,0,0,0,0,0,0
5,p22061_s005,s005_barcode327849,subjectidCC0259,iridCR157,date20200928,p22061-s005_barcode327849-subjectidCC0259-irid...,3,37,12,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
prefixes = {
    'subjectid': 'subjectid',
    'irid': 'irid',
    'date': 'date'
}
for column, prefix in prefixes.items():
    rawdata[column] = rawdata[column].str.replace(f'^{prefix}', '', regex=True)
rawdata['barcode'] = rawdata['barcode'].str[12:] # buts the sxxx_barcode prefix
rawdata.head()

,sample_id,barcode,subjectid,irid,date,gene_id,ENSG00000223972,ENSG00000227232,ENSG00000278267,ENSG00000243485,...,GU280_gp02_S,GU280_gp03_ORF3a,GU280_gp04_E,GU280_gp05_M,GU280_gp06_ORF6,GU280_gp07_ORF7a,GU280_gp08_ORF7b,GU280_gp09_ORF8,GU280_gp10_N,GU280_gp11_ORF10
1,p22061_s001,269931,CC0069,CR141,20200605,p22061-s001_barcode269931-subjectidCC0069-irid...,0,28,30,0,...,0,0,0,0,0,0,0,0,0,0
2,p22061_s002,270483,CC0022,CR074,20200609,p22061-s002_barcode270483-subjectidCC0022-irid...,0,4,21,0,...,0,0,0,0,0,0,0,0,0,0
3,p22061_s003,288558,CC0138,CR069,20200715,p22061-s003_barcode288558-subjectidCC0138-irid...,1,6,14,0,...,0,0,0,0,0,0,0,0,0,0
4,p22061_s004,322576,CC0256,CR169,20200923,p22061-s004_barcode322576-subjectidCC0256-irid...,3,11,15,0,...,0,0,0,0,0,0,0,0,0,0
5,p22061_s005,327849,CC0259,CR157,20200928,p22061-s005_barcode327849-subjectidCC0259-irid...,3,37,12,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# Convert the date_column to datetime format so i can hopefully merge them
metadata['PBMC.Collection.Date'] = pd.to_datetime(metadata['PBMC.Collection.Date'], format='%m/%d/%y')
metadata['PBMC.Collection.Date'] = metadata['PBMC.Collection.Date'].dt.strftime('%Y%m%d')


In [9]:
# I now try to create a new index to merge meta with rawdata
metadata['merge_index'] = metadata['PBMC.Collection.Date'] + metadata['Subject.ID']
print(metadata.head())

rawdata['merge_index'] = rawdata['date'] + rawdata['subjectid']
rawdata.head()

  Subject.ID PBMC.Collection.Date  Age at PBMC Collection (Months)     Sex  \
0     CC0014             20201015                             10.7    Male   
1     CC0014             20210316                             15.6    Male   
2     CC0016             20201215                             12.8  Female   
3     CC0016             20210701                             19.3  Female   
4     CC0022             20200609                              7.0  Female   

   Diff.From.Infant.Pos disStage     merge_index  
0                 -82.0      pre  20201015CC0014  
1                  70.0     conv  20210316CC0014  
2                 -58.0      pre  20201215CC0016  
3                 140.0     conv  20210701CC0016  
4                -153.0      pre  20200609CC0022  


,sample_id,barcode,subjectid,irid,date,gene_id,ENSG00000223972,ENSG00000227232,ENSG00000278267,ENSG00000243485,...,GU280_gp03_ORF3a,GU280_gp04_E,GU280_gp05_M,GU280_gp06_ORF6,GU280_gp07_ORF7a,GU280_gp08_ORF7b,GU280_gp09_ORF8,GU280_gp10_N,GU280_gp11_ORF10,merge_index
1,p22061_s001,269931,CC0069,CR141,20200605,p22061-s001_barcode269931-subjectidCC0069-irid...,0,28,30,0,...,0,0,0,0,0,0,0,0,0,20200605CC0069
2,p22061_s002,270483,CC0022,CR074,20200609,p22061-s002_barcode270483-subjectidCC0022-irid...,0,4,21,0,...,0,0,0,0,0,0,0,0,0,20200609CC0022
3,p22061_s003,288558,CC0138,CR069,20200715,p22061-s003_barcode288558-subjectidCC0138-irid...,1,6,14,0,...,0,0,0,0,0,0,0,0,0,20200715CC0138
4,p22061_s004,322576,CC0256,CR169,20200923,p22061-s004_barcode322576-subjectidCC0256-irid...,3,11,15,0,...,0,0,0,0,0,0,0,0,0,20200923CC0256
5,p22061_s005,327849,CC0259,CR157,20200928,p22061-s005_barcode327849-subjectidCC0259-irid...,3,37,12,0,...,0,0,0,0,0,0,0,0,0,20200928CC0259


In [10]:
combi_df = pd.merge(metadata, rawdata, on='merge_index', how='inner')
print(combi_df.shape)

## convert age 
combi_df['Age (years)'] = combi_df['Age at PBMC Collection (Months)']  / 12.
combi_df.head()

(119, 60708)


,Subject.ID,PBMC.Collection.Date,Age at PBMC Collection (Months),Sex,Diff.From.Infant.Pos,disStage,merge_index,sample_id,barcode,subjectid,...,GU280_gp03_ORF3a,GU280_gp04_E,GU280_gp05_M,GU280_gp06_ORF6,GU280_gp07_ORF7a,GU280_gp08_ORF7b,GU280_gp09_ORF8,GU280_gp10_N,GU280_gp11_ORF10,Age (years)
0,CC0014,20201015,10.7,Male,-82.0,pre,20201015CC0014,p22061_s008,340540,CC0014,...,0,0,0,0,0,0,0,0,0,0.891667
1,CC0014,20210316,15.6,Male,70.0,conv,20210316CC0014,p22061_s049,444946,CC0014,...,0,0,0,0,0,0,0,0,0,1.300000
2,CC0016,20201215,12.8,Female,-58.0,pre,20201215CC0016,p22061_s077,381632,CC0016,...,0,0,0,0,0,0,0,0,0,1.066667
3,CC0016,20210701,19.3,Female,140.0,conv,20210701CC0016,p22061_s090,513755,CC0016,...,0,0,0,0,0,0,0,0,0,1.608333
4,CC0022,20200609,7.0,Female,-153.0,pre,20200609CC0022,p22061_s002,270483,CC0022,...,0,0,0,0,0,0,0,0,0,0.583333


In [11]:
combi_df.to_csv(f'/home/eliasschreiner/PROJEKT/2023_Wimmers_CoVid_multi-omics/2023_Wimmers_combi_df.csv')

## As there was a confusion with PBMC vs. whole blood datatype (PBMC stated in col header and whole blood in https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE239787) i also want to import the raw counts from GEO 


In [12]:
raw_counts_GEO_df = pd.read_csv(f"{root_folder}/raw_counts_GSE239787.txt", sep="\t")
print(raw_counts_GEO_df.shape)
raw_counts_GEO_df.head()

(60695, 91)


,"x[, 1]",p22061.s001,p22061.s002,p22061.s003,p22061.s004,p22061.s005,p22061.s006,p22061.s007,p22061.s008,p22061.s009,...,p22061.s112,p22061.s113,p22061.s114,p22061.s115,p22061.s116,p22061.s117,p22061.s118,p22061.s119,p22061.s120,p22061.s121
0,ENSG00000223972,0,0,1,3,3,2,0,2,0,...,1,0,0,0,1,2,9,0,3,0
1,ENSG00000227232,28,4,6,11,37,4,0,3,12,...,7,4,6,7,0,6,6,5,6,5
2,ENSG00000278267,30,21,14,15,12,20,17,18,19,...,26,30,23,25,6,18,25,14,37,21
3,ENSG00000243485,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ENSG00000284332,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
import sys
sys.path.append(os.path.abspath(os.path.join('..', 'utils')))
import geo_utils
from geo_utils import read_prefixes_from_metadata_txt


metadata_GEO_df = read_prefixes_from_metadata_txt(f"{root_folder}/GSE239787_series_matrix.txt")
metadata_GEO_df[['ID', 'Status']] = metadata_GEO_df['title'].str.replace('Subject ', '').str.split(' ', expand=True)
print(metadata_GEO_df.shape)


(90, 44)


/home/eliasschreiner/PROJEKT/utils/geo_utils.py:46: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip('"') if isinstance(x, str) else x)


In [21]:
columns_metadata = ['geo_accession', 'source_name_ch1', 'instrument_model', 'library_strategy', 'platform_id', 'ID', 'Status',
                    'characteristics_ch1.2', 'characteristics_ch1.3', 'characteristics_ch1.4', 'contact_city', 'description']
metadata_GEO_df_reduced_metadata = metadata_GEO_df[columns_metadata]
metadata_GEO_df_reduced_metadata.head()

,geo_accession,source_name_ch1,instrument_model,library_strategy,platform_id,ID,Status,characteristics_ch1.2,characteristics_ch1.3,characteristics_ch1.4,contact_city,description
0,GSM7673201,Whole blood,Illumina NovaSeq 6000,RNA-Seq,GPL24676,CC0069,healthy,subject: CC0069,Sex: Male,"age, months: 5.4",Palo Alto,p22061.s001
1,GSM7673202,Whole blood,Illumina NovaSeq 6000,RNA-Seq,GPL24676,CC0022,healthy,subject: CC0022,Sex: Female,"age, months: 7",Palo Alto,p22061.s002
2,GSM7673203,Whole blood,Illumina NovaSeq 6000,RNA-Seq,GPL24676,CC0138,healthy,subject: CC0138,Sex: Female,"age, months: 4.6",Palo Alto,p22061.s003
3,GSM7673204,Whole blood,Illumina NovaSeq 6000,RNA-Seq,GPL24676,CC0256,healthy,subject: CC0256,Sex: Male,"age, months: 1.6",Palo Alto,p22061.s004
4,GSM7673205,Whole blood,Illumina NovaSeq 6000,RNA-Seq,GPL24676,CC0259,healthy,subject: CC0259,Sex: Male,"age, months: 1.7",Palo Alto,p22061.s005


In [25]:
raw_counts_GEO_df.T 
raw_counts_transposed_df = raw_counts_GEO_df.set_index('x[, 1]').T.reset_index().rename(columns={'index': 'key'})
raw_counts_transposed_df.head()

"x[, 1]",key,ENSG00000223972,ENSG00000227232,ENSG00000278267,ENSG00000243485,ENSG00000284332,ENSG00000237613,ENSG00000268020,ENSG00000240361,ENSG00000186092,...,GU280_gp02_S,GU280_gp03_ORF3a,GU280_gp04_E,GU280_gp05_M,GU280_gp06_ORF6,GU280_gp07_ORF7a,GU280_gp08_ORF7b,GU280_gp09_ORF8,GU280_gp10_N,GU280_gp11_ORF10
0,p22061.s001,0,28,30,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,p22061.s002,0,4,21,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,p22061.s003,1,6,14,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,p22061.s004,3,11,15,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,p22061.s005,3,37,12,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
metadata_GEO_df_reduced_metadata = metadata_GEO_df_reduced_metadata.rename(columns={'description':'key'})
merged_df_GSE239787 = pd.merge(metadata_GEO_df_reduced_metadata, raw_counts_transposed_df, on='key')
print(merged_df_GSE239787.shape)
merged_df_GSE239787.head()

(90, 60707)


,geo_accession,source_name_ch1,instrument_model,library_strategy,platform_id,ID,Status,characteristics_ch1.2,characteristics_ch1.3,characteristics_ch1.4,...,GU280_gp02_S,GU280_gp03_ORF3a,GU280_gp04_E,GU280_gp05_M,GU280_gp06_ORF6,GU280_gp07_ORF7a,GU280_gp08_ORF7b,GU280_gp09_ORF8,GU280_gp10_N,GU280_gp11_ORF10
0,GSM7673201,Whole blood,Illumina NovaSeq 6000,RNA-Seq,GPL24676,CC0069,healthy,subject: CC0069,Sex: Male,"age, months: 5.4",...,0,0,0,0,0,0,0,0,0,0
1,GSM7673202,Whole blood,Illumina NovaSeq 6000,RNA-Seq,GPL24676,CC0022,healthy,subject: CC0022,Sex: Female,"age, months: 7",...,0,0,0,0,0,0,0,0,0,0
2,GSM7673203,Whole blood,Illumina NovaSeq 6000,RNA-Seq,GPL24676,CC0138,healthy,subject: CC0138,Sex: Female,"age, months: 4.6",...,0,0,0,0,0,0,0,0,0,0
3,GSM7673204,Whole blood,Illumina NovaSeq 6000,RNA-Seq,GPL24676,CC0256,healthy,subject: CC0256,Sex: Male,"age, months: 1.6",...,0,0,0,0,0,0,0,0,0,0
4,GSM7673205,Whole blood,Illumina NovaSeq 6000,RNA-Seq,GPL24676,CC0259,healthy,subject: CC0259,Sex: Male,"age, months: 1.7",...,0,0,0,0,0,0,0,0,0,0


In [30]:
merged_df_GSE239787.to_csv(f'/home/eliasschreiner/PROJEKT/2023_Wimmers_CoVid_multi-omics/2023_Wimmers_GSE239787_GEO.csv')

In [19]:
# um die zu matchen brauch ich nun die p22061_BatchA-B-RNAQC.xlsx ...  okay, die arbeit war wohl umsonst: description im anderen dataset sagt genau das was wir wollen ... 
metadata_xlsx = pd.read_excel(f'{root_folder}/p22061_BatchA-B_RNAQC.xlsx')
metadata_xlsx["Sample ID"] = metadata_xlsx["Sample ID"].str.replace('_', '.')
sample_name_pattern = r'barcode(?P<barcode>\d+)-subjectid(?P<subjectid>\w+)-irid(?P<irid>\w+)-date(?P<date>[\d\-]+)'
metadata_xlsx_sample_name = metadata_xlsx['Sample Name'].str.extract(sample_name_pattern)
metadata_xlsx = pd.concat([metadata_xlsx, metadata_xlsx_sample_name], axis=1)
metadata_xlsx.head()

,Sample ID,Sample Name,Match Group,Disease Stage,Library Prep Batch,Extraction Date,RIN,Qubit Concentration (ng/ul),Yield (ng),DV 200,Unnamed: 10,barcode,subjectid,irid,date
0,p22061.s008,barcode340540-subjectidCC0014-iridCR133-date20...,CC0014,pre,A,2022-03-03,5.3,83.2,7072.0,93.31,NaN,340540,CC0014,CR133,20201015
1,p22061.s036,barcode543269-subjectidCC0173-iridna-date20210818,CC0014,matched-ctrl,A,2022-03-04,4.5,130.0,11050.0,93.75,NaN,543269,CC0173,na,20210818
2,p22061.s049,barcode444946-subjectidCC0014-iridCR133-date20...,CC0014,conv,A,2022-03-04,1.0,31.5,2677.5,76.78,NaN,444946,CC0014,CR133,20210316
3,p22061.s002,barcode270483-subjectidCC0022-iridCR074-date20...,CC0022,pre,A,2022-03-03,4.9,154.0,13090.0,93.92,NaN,270483,CC0022,CR074,20200609
4,p22061.s012,barcode370604-subjectidCC0022-iridCR074-date20...,CC0022,acute,A,2022-03-03,4.0,118.0,10030.0,93.48,NaN,370604,CC0022,CR074,20201202
